In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
import numpy as np
from typing_extensions import Self
from typing import Any
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, MultiTaskElasticNetCV, LassoLars, BayesianRidge, ARDRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score  
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import random
from collections import Counter
from app.pipelines import SalaryPredictionPipeline

In [3]:
df = pd.read_csv(r"C:\Users\tomas\OneDrive\Desktop\Proyectos\Vanguard\GET_YOUR_SALARY\data\tech_salaries_filtered_no_others.csv")
df['contrato'] = df['contrato'].replace({
    'Tercerizado (trabajo a través de consultora o agencia)': 'Contractor',
    'Freelance': 'Contractor',
    "Participación societaria en una cooperativa": 'Contractor',
})
#df = df[df["dedicacion"] != "Part-Time"]
df["marvin_rol"] = df["marvin_rol"].astype("category")


In [4]:
#df.drop(columns=["dedicacion"], inplace=True)

In [5]:
def remove_outliers_iqr(df, num_columns, threshold=1.5):
    df_clean = df.copy()
    
    # Contador de filas originales
    rows_original = len(df_clean)
    
    # Máscara para mantener registro de filas válidas
    valid_rows = pd.Series(True, index=df_clean.index)
    
    # Detectar outliers en cada columna
    for col in num_columns:
        # Calcular cuartiles y IQR
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Definir límites para outliers
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        # Crear máscara para esta columna
        col_mask = (df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)
        
        # Contar outliers en esta columna
        outliers_count = (~col_mask).sum()
        print(f"Columna '{col}': {outliers_count} outliers detectados")
        print(f"  - Límite inferior: {lower_bound:.2f}")
        print(f"  - Límite superior: {upper_bound:.2f}")
        
        # Actualizar máscara de filas válidas
        valid_rows = valid_rows & col_mask
    
    # Aplicar la máscara para eliminar filas con outliers
    df_clean = df_clean[valid_rows]
    
    # Mostrar estadísticas
    rows_removed = rows_original - len(df_clean)
    print(f"\nSe eliminaron {rows_removed} filas con outliers ({rows_removed/rows_original:.2%} del dataset)")
    print(f"Dataset original: {rows_original} filas")
    print(f"Dataset limpio: {len(df_clean)} filas")
    
    return df_clean

In [6]:
df = remove_outliers_iqr(df,['salario', 'anos_de_experiencia', 'antiguedad_en_la_empresa_actual','edad'])

Columna 'salario': 60 outliers detectados
  - Límite inferior: -1018862.80
  - Límite superior: 5964771.33
Columna 'anos_de_experiencia': 90 outliers detectados
  - Límite inferior: -10.50
  - Límite superior: 25.50
Columna 'antiguedad_en_la_empresa_actual': 357 outliers detectados
  - Límite inferior: -3.50
  - Límite superior: 8.50
Columna 'edad': 45 outliers detectados
  - Límite inferior: 12.50
  - Límite superior: 56.50

Se eliminaron 469 filas con outliers (14.06% del dataset)
Dataset original: 3335 filas
Dataset limpio: 2866 filas


In [7]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Supongamos que tienes un DataFrame con variables mixtas
# df = pd.DataFrame(...)

# 1. Usar codificación one-hot para variables categóricas
df_encoded = pd.get_dummies(df, drop_first=True)
correlation_matrix = df_encoded.corr()

# 2. Otra opción: usar codificación ordinal para variables categóricas
# Ejemplo con una columna categórica 'categoria'
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df_copy = df.copy()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
df_copy[categorical_cols] = encoder.fit_transform(df[categorical_cols])
correlation_ordinal = df_copy.corr()

# 3. Para correlaciones entre variables categóricas y numéricas
# Puedes usar correlación de punto biserial o eta cuadrado

In [8]:
correlation_ordinal

,dedicacion,contrato,salario,anos_de_experiencia,antiguedad_en_la_empresa_actual,anos_en_el_puesto_actual,cuantas_personas_tenes_a_cargo,cantidad_de_personas_en_tu_organizacion,modalidad_de_trabajo,edad,seniority,marvin_rol
dedicacion,1.000000,-0.090503,-0.138414,-0.038291,0.004183,-0.021964,-0.028140,-0.041248,-0.028953,-0.040186,-0.039318,-0.011974
contrato,-0.090503,1.000000,-0.027929,0.043364,0.120939,0.030805,0.056736,0.115487,0.159581,0.049714,0.000982,0.066247
salario,-0.138414,-0.027929,1.000000,0.420535,0.108589,0.117063,0.197124,0.047863,0.046782,0.227416,0.332754,-0.002916
anos_de_experiencia,-0.038291,0.043364,0.420535,1.000000,0.267752,0.392321,0.184530,0.024355,-0.009794,0.608309,0.533967,0.013775
antiguedad_en_la_empresa_actual,0.004183,0.120939,0.108589,0.267752,1.000000,0.313844,0.175339,0.016233,-0.001434,0.180979,0.209474,0.022013
anos_en_el_puesto_actual,-0.021964,0.030805,0.117063,0.392321,0.313844,1.000000,0.056340,-0.017864,-0.022076,0.265801,0.327870,-0.008770
cuantas_personas_tenes_a_cargo,-0.028140,0.056736,0.197124,0.184530,0.175339,0.056340,1.000000,0.003367,0.046654,0.152471,-0.078944,0.033198
cantidad_de_personas_en_tu_organizacion,-0.041248,0.115487,0.047863,0.024355,0.016233,-0.017864,0.003367,1.000000,0.048579,0.018850,0.014635,0.024611
modalidad_de_trabajo,-0.028953,0.159581,0.046782,-0.009794,-0.001434,-0.022076,0.046654,0.048579,1.000000,-0.027840,-0.045201,0.066383
edad,-0.040186,0.049714,0.227416,0.608309,0.180979,0.265801,0.152471,0.018850,-0.027840,1.000000,0.318041,0.067359


In [9]:

X = df.drop('salario', axis=1)
y = df['salario']
y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.15, random_state=42
)

In [10]:
train_set = pd.concat([X_train, y_train], axis=1)

In [11]:
print("Distribución original de seniority:")
print(train_set['seniority'].value_counts())
print("\n")

# Separar características categóricas y numéricas
cat_columns = ['dedicacion','contrato', 'cantidad_de_personas_en_tu_organizacion', 'modalidad_de_trabajo', 'seniority', 'marvin_rol']
num_columns = ['salario', 'anos_de_experiencia', 'antiguedad_en_la_empresa_actual', 
              'anos_en_el_puesto_actual', 'cuantas_personas_tenes_a_cargo', 'edad']

# Para columnas categóricas, guardaremos las frecuencias de cada valor
cat_value_freqs = {}
for col in cat_columns:
    if col != 'seniority':  # Seniority es nuestra variable objetivo
        cat_value_freqs[col] = train_set[col].value_counts(normalize=True).to_dict()

# Preparación para SMOTE: codificar variables categóricas
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_encoded = encoder.fit_transform(train_set[cat_columns])
cat_encoded_train_set = pd.DataFrame(cat_encoded, columns=encoder.get_feature_names_out(cat_columns))

# Combinar con características numéricas
X = pd.concat([train_set[num_columns].reset_index(drop=True), cat_encoded_train_set.reset_index(drop=True)], axis=1)
y = train_set['seniority']

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Distribución después de SMOTE:")
print(pd.Series(y_resampled).value_counts())
print("\n")

# Función para generar valores categóricos sintéticos basados en distribución original
def generate_synthetic_cat_value(column_name):
    options = list(cat_value_freqs[column_name].keys())
    weights = list(cat_value_freqs[column_name].values())
    return random.choices(options, weights=weights)[0]

# Recuperar los datos originales y los sintéticos
# Primero, extraemos los valores numéricos rebalanceados
num_resampled = X_resampled[num_columns].copy()

# Ahora necesitamos recuperar las variables categóricas originales
# Decodificar las variables one-hot encoded
cat_columns_encoded = [col for col in X_resampled.columns if col not in num_columns]

# Crear un dataframe balanceado
train_set_balanced = pd.DataFrame()

# Añadir columnas numéricas
for col in num_columns:
    train_set_balanced[col] = num_resampled[col].values

# Añadir columna objetivo
train_set_balanced['seniority'] = y_resampled

# Generar valores sintéticos para columnas categóricas (excepto seniority)
for col in cat_columns:
    if col != 'seniority':
        synthetic_values = []
        for i in range(len(train_set_balanced)):
            if i < len(train_set):  # Para los registros originales, mantener valores originales
                synthetic_values.append(train_set[col].iloc[i])
            else:  # Para registros sintéticos, generar valores basados en la distribución
                synthetic_values.append(generate_synthetic_cat_value(col))
        train_set_balanced[col] = synthetic_values

# Mostrar el dataframe balanceado
print("Datos balanceados con valores sintéticos:")
print(train_set_balanced)
print("\n")

# Verificar la distribución de las variables categóricas
print("Distribución de variables categóricas en datos sintéticos:")
for col in cat_columns:
    if col != 'seniority':
        print(f"\n{col}:")
        print(train_set_balanced[col].value_counts())

# Estadísticas descriptivas de variables numéricas
print("\n\nEstadísticas descriptivas de variables numéricas:")
print(train_set_balanced[num_columns].describe())

Distribución original de seniority:
seniority
Semi-Senior         890
Senior              851
Junior              420
Manager or Above    275
Name: count, dtype: int64


Distribución después de SMOTE:
seniority
Senior              890
Manager or Above    890
Junior              890
Semi-Senior         890
Name: count, dtype: int64


Datos balanceados con valores sintéticos:
        salario  anos_de_experiencia  antiguedad_en_la_empresa_actual  \
0     13.847870                    8                                5   
1     14.756199                   20                                0   
2     13.779884                    0                                0   
3     13.910822                    3                                3   
4     13.815512                    2                                2   
...         ...                  ...                              ...   
3555  14.743067                    7                                0   
3556  14.803797                   14   

In [12]:
X_train = train_set_balanced.drop('salario', axis=1)
y_train = train_set_balanced['salario']

In [13]:
from typing import Any

def get_metrics_by_model(model: Any, use_original_scale: bool = True) -> None:
    pipeline = SalaryPredictionPipeline(model=model)
    model_pipeline = pipeline.build_pipeline()
    
    # Validación cruzada para modelos no lineales
    cv_scores = cross_val_score(
        model_pipeline,
        X_train,
        y_train,
        cv=5,
        scoring='neg_root_mean_squared_error'
    )
    
    model_pipeline_fitted = model_pipeline.fit(X_train, y_train)
    y_pred = model_pipeline_fitted.predict(X_test)

    # Cálculo de métricas mejorado
    metrics = {
        'cv_rmse_mean': abs(cv_scores.mean()),
        'cv_rmse_std': cv_scores.std(),
        'r2': model_pipeline_fitted.score(X_test, y_test),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred))
    }
    
    if use_original_scale:
        y_test_orig = np.expm1(y_test)
        y_pred_orig = np.expm1(y_pred)
        metrics.update({
            'r2_original': r2_score(y_test_orig, y_pred_orig),
            'rmse_original': np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
        })
    
    print(f"\nMétricas para {model.__class__.__name__}:")
    print(f"CV RMSE: {metrics['cv_rmse_mean']:.4f} (±{metrics['cv_rmse_std']:.4f})")
    print(f"Test R²: {metrics['r2']:.4f}")
    print(f"Test RMSE: {metrics['rmse']:.4f}")
    if use_original_scale:
        print(f"Test R² (original): {metrics['r2_original']:.4f}")
        print(f"Test RMSE (original): {metrics['rmse_original']:.2f}")

In [14]:
gbr = GradientBoostingRegressor(n_estimators=200, max_depth=3, subsample=0.6424202471887338, learning_rate=0.03102740950912839)
get_metrics_by_model(gbr)

c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\User


Métricas para GradientBoostingRegressor:
CV RMSE: 0.3738 (±0.0292)
Test R²: 0.3601
Test RMSE: 0.3925
Test R² (original): 0.2980
Test RMSE (original): 1001168.33


c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\User

In [15]:
from app.pipelines import SalaryPredictionPipeline

In [16]:
gbr = GradientBoostingRegressor(n_estimators=200, max_depth=3, subsample=0.6424202471887338, learning_rate=0.03102740950912839)
pipeline = SalaryPredictionPipeline(gbr)
pipeline.build_pipeline()
pipeline.fit(X_train, y_train)

In [17]:
import joblib
from pathlib import Path

# Guardar todo el objeto pipeline
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(exist_ok=True)

joblib.dump(pipeline, MODEL_PATH / "salary_pipeline_v2.pkl")

# Opcional: Guardar metadata adicional
metadata = {
    "model_version": "1.0",
    "training_date": "2025-04-19",
    "features": pipeline.numerical_columns + pipeline.categorical_columns
}

joblib.dump(metadata, MODEL_PATH / "metadata_v2.pkl")

['models\\metadata_v2.pkl']

### Deep Learning (ANASHEI)

In [18]:
# Asumiendo que X_train es un DataFrame de pandas y y_train es una Serie
# Inicializar y entrenar la pipeline de preprocesamiento
preprocessor = SalaryPredictionPipeline()
preprocessor.build_pipeline().fit(X_train, y_train)

# Transformar datos
X_train_processed = preprocessor.transform(X_train)

c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Arquitectura del modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_processed.shape[1],), kernel_initializer='he_normal'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Salida para regresión
])

# Compilación
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mse']
)

# Early Stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Entrenar el modelo (usando el mismo set para train/val por tamaño de datos)
history = model.fit(
    X_train_processed,
    y_train,
    epochs=200,
    batch_size=2,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Métricas finales
train_loss, train_mae, train_mse = model.evaluate(X_train_processed, y_train, verbose=0)
print(f"\nMétricas Finales:")
print(f"- Train MAE: {train_mae:.4f}")
print(f"- Train MSE: {train_mse:.4f}")

Epoch 1/200
 575/1424 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 62.2599 - mae: 6.0785 - mse: 62.2599

KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score


def build_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='mse',
        metrics=['mae']
    )
    return model

def evaluate_deep_model(X_train, y_train, X_test, y_test, use_original_scale=True):
    # 1. Preprocesamiento inicial
    preprocessor = SalaryPredictionPipeline()
    preprocessor.build_pipeline().fit(X_train, y_train)
    X_train_processed = preprocessor.transform(X_train)
    input_dim = X_train_processed.shape[1]
    
    # 2. Cross-validation para RMSE
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_rmse_scores = []
    
    for train_idx, val_idx in kf.split(X_train_processed):
        X_tr, X_val = X_train_processed[train_idx], X_train_processed[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = build_model(input_dim)
        model.fit(X_tr, y_tr, epochs=100, batch_size=16, verbose=0)
        
        y_pred = model.predict(X_val).flatten()
        cv_rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))
    
    # 3. Entrenamiento final y evaluación en test
    model = build_model(input_dim)
    history = model.fit(
        X_train_processed, 
        y_train,
        epochs=200,
        batch_size=32,
        verbose=0,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=15)]
    )
    
    # Preprocesar test
    X_test_processed = preprocessor.transform(X_test)
    y_pred = model.predict(X_test_processed).flatten()
    
    # 4. Calcular métricas
    metrics = {
        'cv_rmse_mean': np.mean(cv_rmse_scores),
        'cv_rmse_std': np.std(cv_rmse_scores),
        'r2': r2_score(y_test, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred))
    }
    
    if use_original_scale:
        y_test_orig = np.expm1(y_test)  # Asumiendo que usaste log1p
        y_pred_orig = np.expm1(y_pred)
        metrics.update({
            'r2_original': r2_score(y_test_orig, y_pred_orig),
            'rmse_original': np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
        })
    
    # 5. Imprimir resultados
    print("\nMétricas del Modelo de Deep Learning:")
    print(f"CV RMSE: {metrics['cv_rmse_mean']:.4f} (±{metrics['cv_rmse_std']:.4f})")
    print(f"Test R²: {metrics['r2']:.4f}")
    print(f"Test RMSE: {metrics['rmse']:.4f}")
    if use_original_scale:
        print(f"Test R² (original): {metrics['r2_original']:.4f}")
        print(f"Test RMSE (original): {metrics['rmse_original']:.2f}")
    
    return metrics, model

In [ ]:
# Suponiendo que tienes X_train, y_train, X_test, y_test
metrics, trained_model = evaluate_deep_model(
    X_train, 
    np.log1p(y_train),  # Si usas logaritmo natural sin +1, cambia a np.log()
    X_test,
    np.log1p(y_test),
    use_original_scale=True
)

c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tomas\.virtualenvs\GET_YOUR_SALARY-sYgm8OCH\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Métricas del Modelo de Deep Learning:
CV RMSE: 0.0376 (±0.0096)
Test R²: -3.8248
Test RMSE: 0.0689
Test R² (original): -17.0522
Test RMSE (original): 2.08
